<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import math

def log2int(x):
  return(math.frexp(x)[1] - 1)

In [6]:
log2int(41)

5

In [17]:
def length(n):
  return(log2int(n+1))

def index(n,m):
  a = (n+1)%(2**(m+1))
  b = 2**m
  if a < b:
    return(0)
  else:
    return(1)
      
            

In [21]:
n = 101
k = length(n)
for i in range(k):
  print(i,index(n,i))

0 0
1 1
2 1
3 0
4 0
5 1


In [22]:
n = 102
k = length(n)
for i in range(k):
  print(i,index(n,i))

0 1
1 1
2 1
3 0
4 0
5 1


In [15]:
7%0

ZeroDivisionError: ignored